In [1]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

In [6]:
df = pd.read_csv("../data/urban_cards_estandar_final.csv")
df.head()

,name,edition,condition,language,price_eur,quantity,total_value,id_carta
0,Acechadora de Llanowar,Dominaria Unida,NM,Español,0.05,2,0.10,11
1,Acuífero contaminado,Dominaria Unida,NM,Español,0.90,1,0.90,20
2,Afanar,Dominaria Unida,NM,Español,0.10,2,0.20,23
3,Afluente fundido,Dominaria Unida,NM,Español,0.25,4,1.00,24
4,Alas rotas,Dominaria Unida,NM,Español,0.05,3,0.15,36


In [17]:
df_competencia = pd.read_csv("../data/competencia_estandar_final.csv")
df_competencia.head()

,name,edition,condition,language,price_eur,quantity,seller,total_value,id_carta
0,Bendición de la furia de batalla,Dominaria Unida,NM,Inglés,0.20,8,Mazvigosl,1.60,138
1,Bosque (V.1),Dominaria Unida,NM,Inglés,0.12,8,Mazvigosl,0.96,151
2,Bosque (V.3),Dominaria Unida,NM,Inglés,0.12,8,Mazvigosl,0.96,153
3,"Radha, señora guerrera de la Coalición",Dominaria Unida,NM,Inglés,0.12,8,Mazvigosl,0.96,871
4,Trasgo germinante,Dominaria Unida,NM,Inglés,0.12,8,Mazvigosl,0.96,1052


In [13]:
df = df.astype({
    "id_carta": int,
    "name": str,
    "edition": str,
    "condition": str,
    "language": str,
    "price_eur": float,
    "quantity": int,
    "total_value": float
})

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1506 entries, 0 to 1505
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1506 non-null   object 
 1   edition      1506 non-null   object 
 2   condition    1506 non-null   object 
 3   language     1506 non-null   object 
 4   price_eur    1506 non-null   float64
 5   quantity     1506 non-null   int64  
 6   total_value  1506 non-null   float64
 7   id_carta     1506 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 94.2+ KB


In [2]:
import psycopg2

# Configurar la conexión
conn = psycopg2.connect(
    dbname="pf_mtg",
    user="luisjimse",
    password="admin",
    host="localhost",
    port="5432"
)

# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

# Verificar la conexión
cur.execute("SELECT version();")
print(cur.fetchone())

# Cerrar la conexión
cur.close()
conn.close()

('PostgreSQL 14.14 (Homebrew) on aarch64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.4), 64-bit',)


In [15]:
# Reabrir la conexión
conn = psycopg2.connect(
    dbname="pf_mtg",
    user="luisjimse",
    password="admin",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Convertir DataFrame a lista de tuplas
records = df.to_records(index=False)
records_list = [
    (
        int(row['id_carta']) if not pd.isna(row['id_carta']) else None,
        str(row['name']),
        str(row['edition']),
        str(row['condition']) if not pd.isna(row['condition']) else None,
        str(row['language']) if not pd.isna(row['language']) else None,
        float(row['price_eur']) if not pd.isna(row['price_eur']) else None,
        int(row['quantity']) if not pd.isna(row['quantity']) else None,
        float(row['total_value']) if not pd.isna(row['total_value']) else None
    )
    for _, row in df.iterrows()
]

# Insertar múltiples registros con ejecuta_many()
query = """
    INSERT INTO urban_cards (id_carta, name, edition, condition, language, price_eur, quantity, total_value)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(query, records_list)

# Guardar cambios
conn.commit()

# Cerrar la conexión
cur.close()
conn.close()
print("Datos insertados correctamente")

Datos insertados correctamente


In [20]:
# Reabrir la conexión
conn = psycopg2.connect(
    dbname="pf_mtg",
    user="luisjimse",
    password="admin",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Convertir DataFrame a lista de tuplas con tipos nativos
records_list = [
    (
        int(row['id_carta']) if not pd.isna(row['id_carta']) else None,
        str(row['name']),
        str(row['edition']),
        str(row['condition']) if not pd.isna(row['condition']) else None,
        str(row['language']) if not pd.isna(row['language']) else None,
        float(row['price_eur']) if not pd.isna(row['price_eur']) else None,
        int(row['quantity']) if not pd.isna(row['quantity']) else None,
        float(row['total_value']) if not pd.isna(row['total_value']) else None,
        str(row['seller']) if not pd.isna(row['seller']) else None
    )
    for _, row in df_competencia.iterrows()
]

# Insertar múltiples registros con ejecuta_many()
query = """
    INSERT INTO competencia (id_carta, name, edition, condition, language, price_eur, quantity, total_value, seller)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(query, records_list)

# Guardar cambios
conn.commit()

# Cerrar la conexión
cur.close()
conn.close()
print("Datos insertados correctamente")


Datos insertados correctamente


In [21]:
unique_cards = pd.concat([df['name'], df_competencia['name']]).drop_duplicates().reset_index(drop=True)
df_cartas = pd.DataFrame({'nombre_carta': unique_cards})
df_cartas['id_carta'] = df_cartas.index + 1  # Generar IDs


In [22]:
unique_editions = pd.concat([df['edition'], df_competencia['edition']]).drop_duplicates().reset_index(drop=True)
df_edicion = pd.DataFrame({'nombre_edicion': unique_editions})
df_edicion['id_edicion'] = df_edicion.index + 1


In [23]:
df['seller'] = 'UrbanCards'
unique_sellers = pd.concat([df['seller'], df_competencia['seller']]).drop_duplicates().reset_index(drop=True)
df_sellers = pd.DataFrame({'nombre_seller': unique_sellers})
df_sellers['id_seller'] = df_sellers.index + 1


In [24]:
df = df.merge(df_cartas, left_on='name', right_on='nombre_carta', how='left') \
       .merge(df_edicion, left_on='edition', right_on='nombre_edicion', how='left') \
       .merge(df_sellers, left_on='seller', right_on='nombre_seller', how='left')

df_competencia = df_competencia.merge(df_cartas, left_on='name', right_on='nombre_carta', how='left') \
                               .merge(df_edicion, left_on='edition', right_on='nombre_edicion', how='left') \
                               .merge(df_sellers, left_on='seller', right_on='nombre_seller', how='left')


In [25]:
conn = psycopg2.connect(
    dbname="pf_mtg",
    user="luisjimse",
    password="admin",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

In [26]:
records_cartas = df_cartas[['id_carta', 'nombre_carta']].to_records(index=False)
cur.executemany("INSERT INTO cartas (id_carta, nombre_carta) VALUES (%s, %s)", records_cartas)


ProgrammingError: can't adapt type 'numpy.int64'

In [27]:
records_cartas = [
    (int(row['id_carta']), str(row['nombre_carta']))
    for _, row in df_cartas.iterrows()
]
cur.executemany("INSERT INTO cartas (id_carta, nombre_carta) VALUES (%s, %s)", records_cartas)


In [28]:
records_edicion = [
    (int(row['id_edicion']), str(row['nombre_edicion']))
    for _, row in df_edicion.iterrows()
]
cur.executemany("INSERT INTO edicion (id_edicion, nombre_edicion) VALUES (%s, %s)", records_edicion)


In [29]:
records_sellers = [
    (int(row['id_seller']), str(row['nombre_seller']))
    for _, row in df_sellers.iterrows()
]
cur.executemany("INSERT INTO sellers (id_seller, nombre_seller) VALUES (%s, %s)", records_sellers)


In [30]:
df_cartas_edicion = pd.concat([df, df_competencia])[['id_carta', 'id_edicion']].drop_duplicates()
records_ce = [
    (int(row['id_carta']), int(row['id_edicion']))
    for _, row in df_cartas_edicion.iterrows()
]
cur.executemany("INSERT INTO cartas_edicion (id_carta, id_edicion) VALUES (%s, %s)", records_ce)


KeyError: "['id_carta'] not in index"

In [31]:
print(df.columns)
print(df_competencia.columns)


Index(['name', 'edition', 'condition', 'language', 'price_eur', 'quantity',
       'total_value', 'id_carta_x', 'seller', 'nombre_carta', 'id_carta_y',
       'nombre_edicion', 'id_edicion', 'nombre_seller', 'id_seller'],
      dtype='object')
Index(['name', 'edition', 'condition', 'language', 'price_eur', 'quantity',
       'seller', 'total_value', 'id_carta_x', 'nombre_carta', 'id_carta_y',
       'nombre_edicion', 'id_edicion', 'nombre_seller', 'id_seller'],
      dtype='object')


In [32]:
df_cartas_edicion = pd.concat([df, df_competencia])[['id_carta_y', 'id_edicion']].drop_duplicates()
df_cartas_edicion.columns = ['id_carta', 'id_edicion']  # Renombramos para que coincida con la tabla

# Convertimos a tuplas
records_ce = [
    (int(row['id_carta']), int(row['id_edicion']))
    for _, row in df_cartas_edicion.iterrows()
]

# Insertamos en la tabla relacional
cur.executemany(
    "INSERT INTO cartas_edicion (id_carta, id_edicion) VALUES (%s, %s)",
    records_ce
)
conn.commit()


In [33]:
def prepare_ofertas(df_ofertas):
    return [
        (
            row['condition'] if pd.notna(row['condition']) else None,
            row['language'] if pd.notna(row['language']) else None,
            float(row['price_eur']) if pd.notna(row['price_eur']) else None,
            int(row['quantity']) if pd.notna(row['quantity']) else None,
            int(row['id_carta']),
            int(row['id_edicion']),
            int(row['id_seller'])
        )
        for _, row in df_ofertas.iterrows()
    ]

records_ofertas = prepare_ofertas(df)
records_ofertas_comp = prepare_ofertas(df_competencia)

query_oferta = """
    INSERT INTO ofertas (condition, language, price, quantity, id_carta, id_edicion, id_seller)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

cur.executemany(query_oferta, records_ofertas)
cur.executemany(query_oferta, records_ofertas_comp)


KeyError: 'id_carta'

In [34]:
def prepare_ofertas(df_ofertas):
    return [
        (
            row['condition'] if pd.notna(row['condition']) else None,
            row['language'] if pd.notna(row['language']) else None,
            float(row['price_eur']) if pd.notna(row['price_eur']) else None,
            int(row['quantity']) if pd.notna(row['quantity']) else None,
            int(row['id_carta_y']),
            int(row['id_edicion']),
            int(row['id_seller'])
        )
        for _, row in df_ofertas.iterrows()
    ]


In [35]:
records_ofertas = prepare_ofertas(df)
records_ofertas_comp = prepare_ofertas(df_competencia)

query_oferta = """
    INSERT INTO ofertas (condition, language, price, quantity, id_carta, id_edicion, id_seller)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(query_oferta, records_ofertas + records_ofertas_comp)
conn.commit()
